In [5]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib

#path = "C:\\Users\\jjudge3\\Downloads\\"
path = ".\\data\\"
#path = "C:\\Users\\RedShirtImaging\\Desktop\\PhotoZ_Jan2021\\PhotoZ_upgrades\\PhotoZ\\"

prog = 5
plot_all = True
date = "6-25-21"
path += date + "\\prog" + str(prog) + "\\"

Using matplotlib backend: Qt5Agg


In [6]:
# (internal) dimensions of the quadrants
'''
"DM2K_2048x512.cfg",
"DM2K_2048x50.cfg",
"DM2K_1024x160.cfg",
"DM2K_1024x80.cfg",
"DM2K_1024x80.cfg",
"DM2K_1024x40.cfg",
"DM2K_1024x30.cfg",
"DM2K_1024x20.cfg"
'''
dimensions = {
    7 : {'height': 512,
         'width': 2048 },
    5 : {'height': 80,
         'width': 1024 },
    4 : {'height': 40,
         'width': 1024 },
    0 : {'height': 20,
         'width': 1024}
}
height = dimensions[prog]['height'] # height of the final image
width = dimensions[prog]['width']
quadrantSize = height * width * 4

def load_image(image_version, quadrantSize, height, delim=' '):
    height *= 2
    final = np.genfromtxt(path + image_version + ".txt", delimiter = delim) # always tab-delim
    raw = np.genfromtxt(path + "raw-" + image_version + ".txt", delimiter = delim)

    raw_img = raw[:quadrantSize,1].reshape(height,-1)
    final_img = final[:quadrantSize // 2,1].reshape(height,-1) # CDS-correct image is half the size. Reset rows were removed

    print("Shaping final image to:",height // 2, quadrantSize // height)
    print(raw_img.shape, final_img.shape)
    return raw_img, final_img
    
def plot_image(raw_img, final_img, image_version, image_no):
    fig = plt.figure()

    ax1 = fig.add_subplot(2, 1, 1)
    ax2 = fig.add_subplot(2, 1, 2)

    ax1.set_title("Raw, RLI frame " + str(image_no))
    ax2.set_title("CDS subtracted then Deinterleave, RLI frame " + str(image_no))

    ax1.imshow(raw_img, aspect='auto', cmap='jet')
    ax2.imshow(final_img[1:,:], aspect='auto', cmap='jet')

    fig.subplots_adjust(hspace = 0.6)

    plt.show()
    plt.savefig('readout-RLI-' + image_version + ".png")

In [7]:
def remapQuadrants(img):
    # Place second half to the right of the first half
    h,w = img.shape

    half1 = img[:h//2,:]
    half2 = img[h//2:,:]
    img = np.zeros((h//2, w*2))
    img[:,:w] = half1
    img[:,w:] = half2
    return img

def normalizeQuadrants(img):
    h,w = img.shape
    img[:h//2,:w//2] = (img[:h//2,:w//2] - np.min(img[:h//2,:w//2])) / np.max(img[:h//2,:w//2])
    img[:h//2,w//2:] = (img[:h//2,w//2:] - np.min(img[:h//2,w//2:])) / np.max(img[:h//2,w//2:])
    img[h//2:,:w//2] = (img[h//2:,:w//2] - np.min(img[h//2:,:w//2])) /  np.max(img[h//2:,:w//2])
    img[h//2:,w//2:] = (img[h//2:,w//2:] - np.min(img[h//2:,w//2:])) /  np.max(img[h//2:,w//2:])
    return img

def plot_remapped(raw_img, final_img, image_version, image_no):
    raw_img_2 = remapQuadrants(raw_img)
    final_img_2 = remapQuadrants(final_img)

    fig = plt.figure()

    ax1 = fig.add_subplot(2, 1, 1)
    ax3 = fig.add_subplot(2, 1, 2)


    ax1.set_title("Raw, Remapped, RLI frame " + str(image_no))
    ax3.set_title("Processed, Remapped, RLI frame " + str(image_no))

    ax1.imshow(raw_img_2, aspect='auto')
    ax3.imshow(final_img_2, aspect='auto')

    fig.subplots_adjust(hspace = 0.6)

    plt.show()
    plt.savefig('readout-RLI-remapped' + image_version + ".png")


In [8]:

plot_all = True
image_no_selected = 350

for image_no in [0, 150, 350, 450]:
    if plot_all or image_no_selected == image_no:
        image_version = "full-out" + str(image_no)
        raw_img, final_img = load_image(image_version, quadrantSize, height)
        plot_image(raw_img, final_img, image_version, image_no)
        plot_remapped(raw_img, final_img, image_version, image_no)
        print("Displayed frame " + str(image_no))

Shaping final image to: 80 2048
(160, 2048) (160, 1024)
Displayed frame 0
Shaping final image to: 80 2048
(160, 2048) (160, 1024)
Displayed frame 150
Shaping final image to: 80 2048
(160, 2048) (160, 1024)
Displayed frame 350
Shaping final image to: 80 2048
(160, 2048) (160, 1024)
Displayed frame 450
